In [1]:
import pandas as pd

# Carregar o arquivo CSV
df = pd.read_csv("crimes_coordenadas.csv", delimiter=',', encoding='latin1')

# Passo 1: Remover duplicatas com base nas colunas especificadas
df_deduplicated = df.drop_duplicates(subset=['Unidade Territorial', 'Latitude', 'Longitude', 'mes_ano'])

# Passo 2: Agrupar e somar os valores desejados
df_grouped = df_deduplicated.groupby(
    ['Unidade Territorial', 'Latitude', 'Longitude', 'mes_ano'], as_index=False
).agg({
    'total_furtos': 'sum',
    'total_roubos': 'sum',
    'registro_ocorrencias': 'sum'
})

# Passo 3: Manter apenas as colunas especificadas
df_final = df_grouped[['Unidade Territorial', 'Latitude', 'Longitude', 'mes_ano', 'total_furtos', 'total_roubos', 'registro_ocorrencias']]

# Exibir o resultado
print(df_final)

# Salvar o DataFrame final em um novo arquivo CSV
df_final.to_csv('crimes_coordenadas_final.csv', index=False)

      Unidade Territorial  Latitude  Longitude  mes_ano  total_furtos  \
0                Abolicao  -22.8875   -43.2944  2003m01            66   
1                Abolicao  -22.8875   -43.2944  2003m02            80   
2                Abolicao  -22.8875   -43.2944  2003m03            67   
3                Abolicao  -22.8875   -43.2944  2003m04            89   
4                Abolicao  -22.8875   -43.2944  2003m05            77   
...                   ...       ...        ...      ...           ...   
42202               Xerem  -22.6167   -43.3044  2024m03            22   
42203               Xerem  -22.6167   -43.3044  2024m04            21   
42204               Xerem  -22.6167   -43.3044  2024m05            23   
42205               Xerem  -22.6167   -43.3044  2024m06            26   
42206               Xerem  -22.6167   -43.3044  2024m07            23   

       total_roubos  registro_ocorrencias  
0               208                   502  
1               232                

In [2]:
import folium

# Criar o mapa
m = folium.Map(location=[-22.9068, -43.1729], zoom_start=12)

for _, row in df_final.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=row["registro_ocorrencias"] / 100,  # Ajustar o tamanho
        color="red",
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['Unidade Territorial']}: {row['registro_ocorrencias']} ocorrências"
    ).add_to(m)

# Salvar e exibir o mapa
#m.save("mapa1.html")
#m  # Apenas para notebooks

In [3]:
from folium.plugins import MarkerCluster

# Criar o mapa
m = folium.Map(location=[-22.9068, -43.1729], zoom_start=12)

# Adicionar clusters
marker_cluster = MarkerCluster().add_to(m)

# Adicionar marcadores com ícones coloridos
for _, row in df_final.iterrows():
    # Lógica de cor baseada no número total de ocorrências
    registro = row["registro_ocorrencias"]
    if registro < 500:
        color = "green"
    elif registro < 1000:
        color = "orange"
    else:
        color = "red"
    
    # Adicionar marcador ao mapa
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        icon=folium.Icon(color=color, icon="info-sign"),
        popup=(
            f"<b>Unidade Territorial:</b> {row['Unidade Territorial']}<br>"
            f"<b>Ano/Mês:</b> {row['mes_ano']}<br>"
            f"<b>Total de Roubos:</b> {row['total_roubos']}<br>"
            f"<b>Total de Furtos:</b> {row['total_furtos']}<br>"
            f"<b>Total de Ocorrências:</b> {row['registro_ocorrencias']}"
        )
    ).add_to(marker_cluster)

# Salvar e exibir o mapa
m.save("mapa2.html")
#m
